In [209]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import scipy
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal

In [210]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [211]:
# Hyper parameters
num_classes = 10 # とりあえず10人で
num_epochs = 30
# batch_size = 128
batch_size = 64
learning_rate = 0.01

# input_size = 496 # 500秒を5秒ウィンドウで
# sequence_length = 1250 # 1セグメントあたりのサンプル数 250Hz * 5s
input_size = 1250
hidden_size = 100 # 論文でいうhidden layerはこれ?
num_layers = 2 # Two-layered lstm?

In [212]:
radar_frame = pd.read_csv('data/radar_01.csv')
wave = radar_frame.to_numpy()
wave = wave.flatten()
len(wave)


1215199

**2000Hz → 250Hzにダウンサンプリング**  
**1人あたりの合計秒数が600秒 = 10分であることを確認**

In [213]:
default_sample_rate = 2000
down_sample_rate = 250
ratio = default_sample_rate // down_sample_rate

wave_dem = signal.decimate(wave, ratio)
len(wave_dem) // 250

607

In [214]:
len(wave_dem)

151900

**5秒(=250Hz×5=1250個)ごとにセグメンテーション**  
**オーバーラップは1.5秒(=250Hz×1.5=375個)**  
**合計で172個の要素数になる模様**

In [215]:
len(wave_dem) // 900

168

In [216]:
remaining = len(wave_dem)
wave_segments = []
n = 0
n_stop = 1250

while n_stop < len(wave_dem):
    n_start = 0 + (1249 - (350 - 1)) * n
    n_stop = n_start + 1250
    tmp = []
    wave_segments.append(wave_dem[n_start:n_stop])
    # print(n)
    n += 1

len(wave_segments)

169

とりあえずリストでセグメンテーションできた？
次は01-30まで全部これにする

In [217]:
df = pd.DataFrame(wave_segments)
df

,0,1,2,3,4,5,6,7,8,9,...,1240,1241,1242,1243,1244,1245,1246,1247,1248,1249
0,-0.000102,-0.000092,-0.000081,-0.000069,-0.000057,-0.000045,-0.000034,-0.000023,-0.000012,-0.000002,...,-0.000074,-0.000080,-0.000086,-0.000092,-0.000096,-0.000100,-0.000103,-0.000105,-0.000106,-0.000106
1,0.000112,0.000103,0.000095,0.000086,0.000079,0.000072,0.000067,0.000062,0.000059,0.000056,...,-0.000136,-0.000133,-0.000130,-0.000128,-0.000126,-0.000125,-0.000125,-0.000125,-0.000127,-0.000129
2,0.000186,0.000186,0.000186,0.000186,0.000186,0.000186,0.000186,0.000187,0.000187,0.000186,...,0.000031,0.000031,0.000031,0.000030,0.000029,0.000028,0.000027,0.000026,0.000025,0.000024
3,0.000025,0.000016,0.000008,0.000002,-0.000003,-0.000007,-0.000010,-0.000012,-0.000013,-0.000014,...,0.000154,0.000153,0.000153,0.000155,0.000157,0.000161,0.000165,0.000171,0.000179,0.000187
4,-0.000077,-0.000079,-0.000082,-0.000085,-0.000087,-0.000090,-0.000093,-0.000095,-0.000097,-0.000100,...,-0.000065,-0.000062,-0.000059,-0.000058,-0.000057,-0.000058,-0.000059,-0.000062,-0.000065,-0.000070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,-0.000018,-0.000023,-0.000028,-0.000032,-0.000037,-0.000041,-0.000044,-0.000048,-0.000051,-0.000054,...,-0.000117,-0.000117,-0.000116,-0.000115,-0.000113,-0.000112,-0.000111,-0.000109,-0.000108,-0.000107
165,-0.000052,-0.000044,-0.000036,-0.000028,-0.000020,-0.000012,-0.000004,0.000003,0.000011,0.000018,...,-0.000042,-0.000038,-0.000034,-0.000030,-0.000026,-0.000023,-0.000019,-0.000016,-0.000012,-0.000009
166,0.000180,0.000178,0.000176,0.000174,0.000172,0.000169,0.000165,0.000160,0.000152,0.000143,...,0.000109,0.000093,0.000075,0.000057,0.000038,0.000019,0.000002,-0.000014,-0.000027,-0.000038
167,-0.000052,-0.000057,-0.000062,-0.000068,-0.000074,-0.000081,-0.000088,-0.000095,-0.000103,-0.000110,...,-0.000091,-0.000093,-0.000095,-0.000096,-0.000097,-0.000097,-0.000097,-0.000096,-0.000096,-0.000094


In [218]:
# 10人分のdataframeを作る
radar_frame_list = []
for i in range(1, 11):
    file_path = "data/radar_%02d.csv" % i
    radar_frame = pd.read_csv(file_path)
    wave = radar_frame.to_numpy().flatten()
    wave_dem = signal.decimate(wave, ratio)
    print(i, len(wave), len(wave_dem)//900)

    remaining = len(wave_dem)
    n = 0
    n_stop = 1250
    wave_segments = []

    while n_stop < len(wave_dem):
        n_start = 0 + (1249 - (350 - 1)) * n
        n_stop = n_start + 1250
        tmp = []
        wave_segments.append(wave_dem[n_start:n_stop])
        n += 1
    
    radar_frame_list.append(wave_segments)

1 1215199 168
2 1244699 172
3 1202699 167
4 1206149 167
5 1220199 169
6 1221849 169
7 1269849 176
8 1237149 171
9 1298799 180
10 1278249 177


**人数分のデータフレームを作成**  
* 行は被験者ごと，列はセグメントごと

In [219]:
print(radar_frame_list[0][0][0])
df = pd.DataFrame(radar_frame_list)
df

-0.00010217575366496895


,0,1,2,3,4,5,6,7,8,9,...,170,171,172,173,174,175,176,177,178,179
0,"[-0.00010217575366496895, -9.226088291837284e-...","[0.00011229699387508095, 0.0001032509100699501...","[0.00018580354782126828, 0.0001857031428586556...","[2.469195076897992e-05, 1.5866761530307943e-05...","[-7.67209241147911e-05, -7.923320450820692e-05...","[-0.0001361980477122986, -0.000129036270940753...","[0.00012517671357017653, 0.0001279797125066038...","[-5.305896864533764e-05, -5.612765612573157e-0...","[-8.081513360944013e-05, -7.385777693628861e-0...","[0.00020484894699947232, 0.0002019216523763631...",...,None,None,None,None,None,None,None,None,None,None
1,"[-7.51486993902038e-05, -8.314413910644068e-05...","[-2.9164517084747145e-05, -2.6524112808993577e...","[6.682297090865066e-05, 7.560042042380256e-05,...","[-0.00010513358067542655, -0.00010161305211377...","[2.725954692718539e-05, 2.569664843079844e-05,...","[-2.1495651373175345e-05, -2.2238639051121963e...","[-0.00013926024027390886, -0.00013638940599371...","[3.8327667046612755e-06, 3.816107957955543e-06...","[-2.882893765247829e-06, -3.29249303967239e-06...","[1.9566993964780114e-05, 1.6866280887701476e-0...",...,"[-0.00016515986122831954, -0.00016574639806197...","[4.5044530274967875e-05, 4.7117912158978656e-0...","[2.356640525322364e-05, 2.1818824919223388e-05...",None,None,None,None,None,None,None
2,"[-0.00010388864679397265, -9.87042938008913e-0...","[9.655852989756008e-05, 9.79198405174223e-05, ...","[1.6744066262912926e-05, 1.9515273404464838e-0...","[-3.5870453145601006e-05, -3.6576533687014696e...","[-3.3414687935717644e-05, -3.4837979737338904e...","[1.3228629144011063e-05, 1.5806069061146607e-0...","[5.084611063167068e-05, 5.1247928947864065e-05...","[1.1331568739055861e-05, 1.1701578401282868e-0...","[8.265009922803249e-05, 8.239057575923266e-05,...","[-1.1570917801348257e-05, -8.362759233598307e-...",...,None,None,None,None,None,None,None,None,None,None
3,"[-4.4885622489585685e-06, -1.459302588458722e-...","[7.827046033008795e-05, 8.069700100674337e-05,...","[0.00013799220568810224, 0.0001375301532834090...","[-7.254389938913604e-05, -7.712668281899455e-0...","[-0.0001232832093546887, -0.000123743303138528...","[-4.87228527028466e-06, -1.8425931096097708e-0...","[-0.00016550555466526127, -0.00016829158455922...","[8.913379185882644e-05, 9.195144527323694e-05,...","[-0.00016853594459161672, -0.00016384435539281...","[0.00014699066879112553, 0.0001345852572133978...",...,None,None,None,None,None,None,None,None,None,None
4,"[-2.3435893667948337e-05, -2.4314464854650993e...","[0.0006279100147601653, 0.000629090281664247, ...","[-0.00012436191708543396, -0.00010959385623465...","[-8.849285242737494e-05, -8.619163949896179e-0...","[-8.645844908562058e-05, -8.456196382448075e-0...","[-0.00012505046443594297, -0.00012799945961583...","[0.00016504781115203883, 0.0001565690715660882...","[7.163609816947924e-05, 7.488307948352697e-05,...","[0.0004291856731311547, 0.00044428052246856724...","[9.488232980574923e-06, 9.328640695779351e-06,...",...,None,None,None,None,None,None,None,None,None,None
5,"[9.461745498843668e-05, 4.4957484964293506e-05...","[1.4446638617456731e-05, 1.0657424118735377e-0...","[4.818525386039256e-05, 4.788968418548144e-05,...","[-1.2851034109091465e-05, -4.589571231272923e-...","[-0.00023655780242741326, -0.00023449833972404...","[4.604103264613258e-05, 4.516991856118031e-05,...","[-0.00014216245275322633, -0.00014124154331185...","[0.00010186620815226912, 0.0001024111741420095...","[-4.826151559564035e-06, -2.5656844342844323e-...","[1.1020931312847036e-05, 1.028644655790929e-05...",...,None,None,None,None,None,None,None,None,None,None
6,"[-2.8948179566577443e-05, -2.3305587669885243e...","[-9.87591118247912e-05, -9.89575740903703e-05,...","[2.746629784017599e-06, 6.294335201669757e-07,...","[-6.474715118223146e-05, -6.602547213139858e-0...","[-0.00021309686240984793, -0.00021374655167428...","[0.0002907191011337922, 0.00030034445980470806...","[-0.0001204787655214717

In [220]:
print(len(radar_frame_list), len(radar_frame_list[0]))

10 169


In [221]:
counter = 0
for label in labels:
    if label == 10:
        counter += 1
counter

178

**Datasetに使うためにlabelをセグメントごとに(セグメントの数だけ)用意**  
* 被験者01と被験者02のセグメントの数は異なるが，きちんと対応できている

In [222]:
labels = []
for i in range(len(radar_frame_list)):
    for j in range(len(radar_frame_list[i])):
        labels.append(i + 1)

labels_df = pd.DataFrame(labels)
labels_df

,0
0,1
1,1
2,1
3,1
4,1
...,...
1718,10
1719,10
1720,10
1721,10


In [223]:
labels_df.values.reshape(-1).shape

(1723,)

In [224]:
tmp = df.to_numpy().reshape(-1)
df = pd.DataFrame(tmp).dropna()
df.shape

(1723, 1)

**radar_frame_listをreshapeして被験者ごとの行をなくす**

In [225]:
class MyDataset(Dataset):
    def __init__(self, dataset, labels, root_dir, transform=None) -> None:
        # super().__init__()
        self.radar_heartbeat = dataset
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform

    
    def __getitem__(self, idx):
        heartbeat_segment = torch.FloatTensor(self.radar_heartbeat[idx])
        subject_label = torch.LongTensor(self.labels[idx])

        return heartbeat_segment, subject_label

    def __len__(self):
        return len(self.radar_heartbeat)

In [226]:
Dataset = MyDataset(df, labels_df, "./data/", transform=transforms.ToTensor())

In [227]:
Dataset

**Train:Testを 8 : 2 に分割**

In [228]:
train_size = int(0.8 * len(df.values))
test_size = len(df.values) - train_size
train_size, test_size

(1378, 345)

In [229]:
train_set, test_set = torch.utils.data.random_split(Dataset, [train_size, test_size])
print(f"full: {len(Dataset)} -> train: {len(train_set)}, test: {len(test_set)}")

full: 1723 -> train: 1378, test: 345


**データローダーを定義**

In [230]:
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

**LSTMクラスを定義**

In [231]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes) -> None:
    super().__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    # self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

    # out, _ = self.rnn(x, h0)
    out, _ = self.lstm(x, (h0, c0))

    out = out[:, -1, :]

    out = self.fc(out)

    return out

In [233]:
model = LSTM(input_size, hidden_size, num_layers, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [234]:
torch.version.cuda

'10.2'